In [3]:
%matplotlib inline
import nibabel as nib
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import sys
sys.path.append('/Users/evaalonsoortiz/Documents/python/susceptibility-to-fieldmap-fft/')
from functions import compute_fieldmap

In [20]:
meas_fmap_path = "../data/fm_Hz.nii.gz"
sim_chi_map_path = "../data/chimap_crop_custom6.nii.gz"
ROIs_map_path = "../data/mask_crop_gre_fov_body_plus_trachea.nii.gz"

In [21]:
def f(x):

    # load simulated susceptibility map in ppm
    chi_img = nib.load(sim_chi_map_path)
    chi_data = chi_img.get_fdata()
    
    # load segmentation labels map
    ROIs_img = nib.load(ROIs_map_path)
    ROIs_data = ROIs_img.get_fdata()
    # find indicies that have label = 256, these voxels should correspond to the spinal cord
    ind_sc = np.where(ROIs_data == 256)
    # find indicies that have label = 7 or 8, these voxels should correspond to the trachea (t) and lungs (l); ind_tl
    ind_tl = np.where((ROIs_data == 7) | (ROIs_data == 8))
    
    # load simulated field map in Hz
    b0meas_img = nib.load(meas_fmap_path)
    b0meas_data = b0meas_img.get_fdata()

    # assign variable "x" to indicies corresponding to trachea/lungs
    chi_data[ind_tl] = x

    # compute the field distribution in ppm for the chi_data distribution using the compute_bz function from the susceptibility-to-fieldmap-fft package
    z =  compute_fieldmap.compute_bz(chi_data)
    # convert field distribution from ppm to Hz (assuming 3T)
    z = z*3*42.58 # deltaB0 [Hz] = deltaB0 [ppm] * B0 [T] * 1e-6 * 42.58*e6 [Hz/T]

    return (np.linalg.norm(b0meas_data[ind_sc] - z[ind_sc]))

In [22]:
min_chi_tl = optimize.fmin(f, 0.3)
min_chi_tl[0]

Optimization terminated successfully.
         Current function value: 1691.787615
         Iterations: 23
         Function evaluations: 46


-4.098515625000012